In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations
from math import sin, cos, sqrt, atan2, radians


## Importar archivos

df_pedidos = pd.read_csv('opt_df_pedidos.csv')
df_costos = pd.read_csv('opt_df_costos.csv')
df_patentes_capacidad = pd.read_csv('opt_df_patentes_capacidad.csv')

In [ ]:
df_pedidos.head()

,Unnamed: 0,ZONA,CANT_TON,CD_RUTA,LATITUD,LONGITUD,HORA_ENTREGA_PROM,REST,LCA,COQUIMBO,PTM,LAMPA
0,0,71,3.6,1,-33.050525,-71.401190,0.0,16.2,1,0,0,0
1,1,71,3.6,1,-33.050525,-71.401190,0.0,16.2,1,0,0,0
2,2,69,5.4,1,-32.740517,-71.487467,0.0,16.2,1,0,0,0
3,3,31,5.4,0,-29.957529,-71.338701,0.0,28.8,1,0,0,0
4,4,53,3.6,1,-32.868263,-70.608080,0.0,16.2,1,0,0,0


In [ ]:
df_costos.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,38,39,40,41,42,43,44,45,46,47
0,0,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,...,5130.0,999999.0,999999.0,999999.0,999999.0,999999.0,5190.0,999999.0,999999.0,999999.0
1,1,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,...,5130.0,999999.0,999999.0,999999.0,999999.0,999999.0,5190.0,999999.0,999999.0,999999.0
2,2,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,...,4336.0,999999.0,999999.0,999999.0,999999.0,999999.0,4629.0,999999.0,999999.0,999999.0
3,3,8796.0,10000.0,8708.0,10595.0,8708.0,10595.0,11000.0,8708.0,8708.0,...,999999.0,10595.0,10595.0,8708.0,10595.0,8796.0,999999.0,10595.0,10595.0,999999.0
4,4,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,999999.0,...,5620.0,999999.0,999999.0,999999.0,999999.0,999999.0,5778.0,999999.0,999999.0,999999.0


In [ ]:
df_patentes_capacidad.head()

,Unnamed: 0,CAP_TON
0,0,28.8
1,1,28.8
2,2,28.8
3,3,28.8
4,4,28.8


In [ ]:
def distance_two_points(coord1, coord2):
    """ Calculate the distance between two points A and B

    :type coord1: tuple(float)
    :param coord1: Coordinates of point A

    :type coord2: tuple(float)
    :param coord2: Coordinates of point B

    :return: Distance beetween point A and point B in kilometers

    """
    earth_radius = 6373.0
    lat1 = radians(coord1[0])
    lon1 = radians(coord1[1])
    lat2 = radians(coord2[0])
    lon2 = radians(coord2[1])
    d_lon = lon2 - lon1
    d_lat = lat2 - lat1
    a_value = sin(d_lat / 2)**2 + cos(lat1) * cos(lat2) * sin(d_lon / 2)**2
    b_value = 2 * atan2(sqrt(a_value), sqrt(1 - a_value))
    distance = earth_radius * b_value
    return distance

class EuclideanGraph():
    """ The EuclideanGraph class represents and euclidean graph

    :ivar edges: Weights between clients
    :ivar nodes: List of clients
    """

    def __init__(self, origin, coords):
        """ Create an euclidean graph

        :type origin: tuple(float)
        :param origin: Coordinate of the plant (the origin)

        :type coords: list of tuple(float)
        :param coords: Coordinates of the clients
        """
        coords = coords + [origin]
        edges = {}
        nodes = []
        for i, current in enumerate(coords):
            nodes.append(i)
            for j, comparer in enumerate(coords):
                if i != j:
                    weight = distance_two_points(current, comparer)
                    edges[i, j] = weight
                    edges[j, i] = weight
        self.nodes = nodes #nombre de los nodos
        self.edges = edges #matriz de distancias entre los clientes
        self.start = self.nodes[-1] #Posiciones de la planta (el inicial)

    def shortest_distance(self, node_list):
        """ Calculate shortest path of a list of nodes from the origin using
        Dijkstra Algorithm.

        :type node_list: tuple(float)
        :param node_list: Coordinates of point A

        :return: The distance of the minimun path

        """
        start = self.start
        unvisited = node_list
        visited = [start]
        weight_list = []

        while unvisited:
            selected_weight = float("inf")
            selected_node = None
            for neighbor in unvisited:
                if self.edges[start, neighbor] < selected_weight:
                    selected_weight = self.edges[start, neighbor]
                    selected_node = neighbor

            weight_list.append(self.edges[visited[-1], selected_node])
            unvisited.remove(selected_node)
            visited.append(selected_node)

        return visited, np.array(weight_list)

def valid_hours_window(poss_path, delivery_hour, max_hour_window):
    """ Evaluate if hours in a truck trip are valid together.

    :type poss_path: list
    :param poss_path: List with client indexes in a possible trip

    :type delivery_hour: list
    :param delivery_hour: List with client delivery hours

    :type max_hour_window: integer
    :param max_hour_window: Maximum hour window accepted

    :return: Flag indicating if it is valid
    """
    hours = sorted(list(set([delivery_hour[j] for j in poss_path if delivery_hour[j] != 0])))

    if len(hours) >= 2:
        for j in range(1, len(hours)):
            if hours[j] - hours[j - 1] > max_hour_window:
                return False
    return True

def get_nearest_rest(cant_ton,rest_list):
    for rest in rest_list:
        if cant_ton <= rest:
            return rest
    return 0 #error

def get_possible_clients_combinations_lca(clients, client_demands_list, client_restrictions, plant_coord, clients_coord, zones, delivery_hour, max_distance, max_hour_window, max_truck_cap, plant, city_list=None):
    """ Get possible clients trip combinations considering the maximum distance and the maximum hour window allowed.

    :type plant_coord: list
    :param plant_coord: List of clients

    :type plant_coord: tuple
    :param plant_coord: Origin plant coordinates

    :type clients_coord: list
    :param clients_coord: List with clients coor

    :type zones: list
    :param zones: List with clients zonesdinates

    :type delivery_hour: list
    :param delivery_hour: List with clients delivery hour

    :type max_distance: integer
    :param max_distance: Maximum distance allowed between clients in a trip

    :type max_hour_window: integer
    :param max_hour_window: Maximum hour window accepted in the same trip

    :type plant: str
    :param plant: Origin cement plant

    :return: List with tuples of valid combinations

    """
    n_clients = len(clients_coord)
    max_len_comb = min(n_clients, 4)
    possible_comb = []
    coord_graph = EuclideanGraph(plant_coord, clients_coord)
    modified_restrictions = []
    for c in clients:
        modified_restrictions.append(client_restrictions[c] if client_demands_list[c] <= client_restrictions[c] else get_nearest_rest(client_demands_list[c],[9,16.2,28.8]))
    for i in range(1,max_len_comb + 1):
        for poss_path in combinations(clients, i):
            if len(poss_path) == 1:
                possible_comb.append(poss_path)
                continue
            sum_ton = sum([client_demands_list[c] for c in list(poss_path)])
            max_ton = min(min([modified_restrictions[c] for c in list(poss_path)]), max_truck_cap)
            if sum_ton > max_ton:
                continue
            valid_hours = valid_hours_window(poss_path, delivery_hour, max_hour_window)
            if not valid_hours:
                continue
            _, shortest_path_edges = coord_graph.shortest_distance(list(poss_path))
            if np.all(shortest_path_edges[1:] <= max_distance):
                possible_comb.append(poss_path)

    return possible_comb

def update_parameters_with_combinations(clients, n_trucks, truck_capacities, client_demands_list, cargo_cost_list, cargo_time_list, transp_cost_mat, clients_trip_time):
    """ Update parameters adding new valid combinations as keys

    :type clients: list
    :param clients: List of groups of clients

    :type n_trucks: int
    :param n_trucks: Number of trucks

    :type client_demands_list: tuple
    :param client_demands_list: List with clients demand

    :type cargo_cost_list: list
    :param cargo_cost_list: List with clients cargo cost

    :type transp_cost_mat: 2d numpy array
    :param transp_cost_mat: Matrix with fare cost

    :return: Client demands, Cargo cost and Transportation cost as dictionaries.
    """
    client_demands = {}
    cargo_cost = {}
    cargo_time = {}
    transp_cost = {}
    for cont, comb in enumerate(clients):
        client_demands[comb] = sum([client_demands_list[i] for i in comb])
        cargo_cost[comb] = sum([cargo_cost_list[i] for i in comb])
        cargo_time[comb] = 25 * 60 + clients_trip_time[cont] + sum([cargo_time_list[i] for i in comb])
        for t in range(n_trucks):
            tons_to_pay = truck_capacities[t] if plant in ('lca','ptm') else max(client_demands[comb], 18) if plant == 'lampa' else max(client_demands[comb], 14.4)
            transp_cost[(t, comb)] = max([transp_cost_mat[t][j] for j in comb]) * tons_to_pay

    return client_demands, cargo_cost, transp_cost, cargo_time

def formato_optimizacion_sacos(df_clientes, df_costos, df_trans, plant):
    """ Format parameters as acceptable cplex input values

    :type client_demands_list: tuple
    :param client_demands_list: List with clients demand

    :type cargo_cost_list: list
    :param cargo_cost_list: List with clients cargo cost

    :type transp_cost_mat: 2d numpy array
    :param transp_cost_mat: Matrix with fare cost

    :type plant: str
    :param plant: Origin cement plant

    :return: Client demands, Cargo cost and Transportation cost as dictionaries.
    """
    # clients basic parameters
    n_clients = len(df_clientes)
    clients = list(range(n_clients))
    client_demands_list = list(df_clientes['CANT_TON'])
    client_restrictions = list(df_clientes['REST'])

    # trucks basic parameters
    n_trucks = len(df_trans)
    trucks = list(range(n_trucks))
    truck_capacities = list(df_trans['CAP_TON'])

    # transportation cost matrix
    transp_cost_mat = np.transpose(df_costos.values)

    # cargo cost
    cargo_cost = list(df_clientes['CD_RUTA'])
    # 1: PP, 0: DC
    cargo_fee = 0 if plant in ('lampa','coquimbo') else 1011
    cargo_cost_list = [cargo_fee * int(cargo_cost[i] == 1) * client_demands_list[i] for i in range(n_clients)]
    # 20 or 0.06 seg/sack * 40 sacks/ton
    manual_unload_time = 10
    automatic_unload_time = 0.06
    sack_per_ton = 40
    cargo_time_list = [(manual_unload_time if int(cargo_cost[i] == 1) else automatic_unload_time) * client_demands_list[i] * sack_per_ton for i in range(n_clients)]

    # getting valid combinations considering hours and coordinates
    plant_coord = None
    if plant == 'lca':
        plant_coord = (-32.783989, -71.1971171)
        get_possible_clients_combinations = get_possible_clients_combinations_lca
    elif plant == 'lampa':
        plant_coord = (-33.200587, -70.890789)
        get_possible_clients_combinations = get_possible_clients_combinations_lampa
    elif plant == 'ptm':
        plant_coord = (-41.5339857, -73.130694)
        get_possible_clients_combinations = get_possible_clients_combinations_ptm
    else: #coquimbo
        plant_coord = (-29.9702383, -71.2700102)
        get_possible_clients_combinations = get_possible_clients_combinations_coquimbo

    clients_coord = list(zip(df_clientes['LATITUD'],df_clientes['LONGITUD']))
    zones = list(df_clientes['ZONA'])
    delivery_hour = list(df_clientes['HORA_ENTREGA_PROM'])
    max_distance_dict = {'lampa': 10, 'coquimbo': 6, 'lca': 25, 'ptm': 25}
    max_distance = max_distance_dict[plant]
    max_hour_window = 3
    max_truck_cap = max(truck_capacities)
    clients = get_possible_clients_combinations(clients, client_demands_list, client_restrictions, plant_coord, clients_coord, zones, delivery_hour, max_distance, max_hour_window, max_truck_cap, plant)
    if plant == 'lampa':
        clients_trip_time = clients[1]
        clients = clients[0]
    else:
        clients_trip_time = [0] * len(clients)

    # updating parameters with combinations
    client_demands, cargo_cost, transp_cost, cargo_time = update_parameters_with_combinations(clients, n_trucks, truck_capacities, client_demands_list,
                                                                                  cargo_cost_list, cargo_time_list, transp_cost_mat, clients_trip_time)

    return trucks, clients, truck_capacities, client_demands, transp_cost, cargo_cost, cargo_time

def add_sink_truck(trucks, truck_capacities, transp_cost, clients):
    sink_truck_id = trucks[-1] + 1
    trucks.append(sink_truck_id)
    truck_capacities.append(1000000)
    for c in clients:
        transp_cost[(sink_truck_id, c)] = 999998

In [ ]:
plant = ''

if len(df_pedidos[df_pedidos['LCA'] == 1]) > 0:
    plant = 'lca'
elif len(df_pedidos[df_pedidos['LAMPA'] == 1]) > 0:
    plant = 'lampa'
elif len(df_pedidos[df_pedidos['PTM'] == 1]) > 0:
    plant = 'ptm'
else:
    plant = 'coquimbo'

print("planta: ",plant)

planta:  lca


In [ ]:
%%time
# Optimization format
n_clients = len(df_pedidos)
trucks, clients, truck_capacities, client_demands, transp_cost, cargo_cost, cargo_time = formato_optimizacion_sacos(df_pedidos, df_costos, df_patentes_capacidad, plant)
add_sink_truck(trucks, truck_capacities, transp_cost, clients)

CPU times: user 6.86 s, sys: 72.9 ms, total: 6.94 s
Wall time: 7.59 s


In [ ]:
len(transp_cost)

517489

In [ ]:
print('trucks', trucks)
print('clients', clients)
print('truck_capacities', truck_capacities)
print('client_demands[clients (1, 5, 11)]', client_demands[(1, 5, 11)])
print('transp_cost[(truck 10, clients (1, 5, 11))]', transp_cost[(10,(1, 5, 11))])
print('cargo_cost[clients (1, 5, 11)]', cargo_cost[(1, 5, 11)])
print('cargo_time[clients (1, 5, 11)]', cargo_time[(1, 5, 11)])

trucks [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48]
clients [(0,), (1,), (2,), (3,), (4,), (5,), (6,), (7,), (8,), (9,), (10,), (11,), (12,), (13,), (14,), (15,), (16,), (17,), (18,), (19,), (20,), (21,), (22,), (23,), (24,), (25,), (26,), (27,), (28,), (29,), (30,), (31,), (32,), (33,), (34,), (35,), (36,), (37,), (38,), (39,), (40,), (41,), (42,), (43,), (44,), (45,), (46,), (47,), (48,), (49,), (50,), (51,), (52,), (53,), (54,), (55,), (56,), (57,), (58,), (59,), (60,), (61,), (62,), (63,), (64,), (65,), (66,), (67,), (68,), (69,), (70,), (71,), (72,), (73,), (74,), (75,), (76,), (77,), (78,), (79,), (80,), (0, 1), (0, 5), (0, 8), (0, 10), (0, 11), (0, 15), (0, 20), (0, 22), (0, 23), (0, 27), (0, 28), (0, 30), (0, 37), (0, 42), (0, 64), (0, 67), (0, 69), (0, 71), (0, 72), (0, 73), (0, 74), (0, 75), (0, 76), (0, 77), (0, 78), (1, 5), (1, 8), (1

In [ ]:
from pulp import *

def prep(trucks, clients, truck_capacities, client_demands, transp_cost, cargo_cost):
  assigns = {}
  for c in trucks:
      for g in clients:
          if client_demands[g] <= truck_capacities[c]:
              assigns[(c,g)] = 1 + cargo_cost[g] + transp_cost[(c,g)]
  print(f"len(clients)*len(trucks)={len(clients)*len(trucks)} len(assigns)={len(assigns)}")
  return assigns

def modelo_ini(n_orders, n_trucks, assigns):
  print(f"n_orders={n_orders}, n_trucks={n_trucks}, n_assigns={len(assigns)}")
  # Modelo de optimizacion
  model = LpProblem("PrAsign", LpMinimize)
  # Variable
  x = LpVariable.dicts('x', assigns, lowBound=0, upBound=1, cat=LpInteger)
  # Funcion objetivo
  model += lpSum([assigns[i]*x[i] for i in assigns])
  # Restricciones
  for i in range(n_orders):
      model += lpSum([(i in g) * x[c,g] for (c,g) in assigns]) == 1
  for i in range(n_trucks):
      model += lpSum([(i == c) * x[c,g] for (c,g) in assigns]) <= 1
  return x, model

def modelo_sol(assigns, x, model):
  # Resolucion
  model.solve()
  # Status
  print("Status = %s" % LpStatus[model.status])
  print("Asignacion optima:")
  print("Camion Pedidos")
  # Solucion
  for (c,g) in assigns:
      if x[c,g].varValue:
          print(c, g)
  # Valor
  print("Objective = %f" % value(model.objective))

In [ ]:
n_orders = len(df_pedidos) # numero de pedidos/clientes
n_trucks = len(trucks)     # numero de camiones
assigns = prep(trucks, clients, truck_capacities, client_demands, transp_cost, cargo_cost)

len(clients)*len(trucks)=517489 len(assigns)=483806


In [ ]:
x, model = modelo_ini(n_orders, n_trucks, assigns)

n_orders=81, n_trucks=49, n_assigns=483806


In [ ]:
%%time
modelo_sol(assigns, x, model)

Status = Optimal
Asignacion optima:
Camion Pedidos
0 (6,)
1 (61,)
3 (46, 47, 48, 49)
5 (79,)
6 (17, 50, 52)
7 (62,)
8 (10, 23, 74, 77)
9 (3, 45)
10 (2, 43, 44, 55)
11 (32, 51)
13 (5, 31)
15 (20, 24)
16 (12,)
18 (8, 57, 58, 67)
19 (9, 25)
20 (21, 34, 59)
21 (35, 53, 76)
23 (27, 71, 73, 75)
24 (14, 54, 69)
25 (30, 63)
26 (26,)
27 (66,)
29 (56,)
30 (22, 38, 72, 78)
31 (36,)
33 (39,)
34 (16, 33)
38 (29, 60)
39 (11, 70, 80)
40 (65,)
41 (18,)
42 (19,)
43 (41,)
44 (13, 68)
45 (1, 15, 28, 64)
46 (7,)
47 (4, 40)
48 (0, 37, 42)
Objective = 246914292.925000
CPU times: user 15.4 s, sys: 817 ms, total: 16.2 s
Wall time: 5min 37s
